In [79]:
import pandas as pd
import datetime

In [80]:
userLog = pd.read_csv("./clean_data/userlog_feature.csv")

In [81]:
userLog['datetime'] = pd.to_datetime(userLog['datetime'])
isTrain = userLog['datetime']<datetime.datetime(2018,2,18)
test = userLog[~isTrain]
userLog = userLog[isTrain]


In [ ]:
#testUserAnswerMerged.to_csv("./clean_data/_test_with_train.csv")

In [82]:
userLog1 = userLog.sort_values("datetime")
userLog1.sample()

,userCode,project_id,requestedDevice,userAgent,pageReferrer,year,month,day,hour,datetime,user_clicked_by_hour,user_clicked_projects_by_hour
273392,e44b7ac9-0953-6c4e-7913-165a3d202054,8353,Mobile,iPhone,HomeWebsite,2018,1,14,18,2018-01-14 18:00:00,3,3


In [83]:
userLog2VecDf = userLog1.groupby("userCode").agg({"project_id":lambda x: list(x)})

In [84]:
userLog2VecDf.sample(10)
#userLog2VecDf.shape

,project_id
userCode,
d91e5972-70f2-5811-61b0-74682aae3439,[3809]
acc768c1-62c2-a6d1-30f5-14f936ce4b28,[1945]
0c7d3ffc-c12e-8924-b43e-db2884c536b0,[8910]
6c2a00e0-1317-5800-722c-8fda3e37b23c,"[3297, 3297, 3297]"
affb4599-bdf1-1a51-509a-4298c97b17d9,[3738]
709e7220-5208-4c39-87d1-4949e17ce2a8,"[4381, 4381, 4381]"
da349e79-e5fe-ce2f-8c36-d2f10717353d,[8935]
53fefdc8-8033-cfa7-0a5f-5a3c4b622d7b,[8112]
80790d8c-f3fd-25a6-7022-e634a716e968,[9373]


In [85]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [98]:
projIdxMap = pd.read_csv("./clean_data/proj2Idx.csv")
getProjIdx = lambda x: projIdxMap.query("project_id=='%s'"%x)['projIdx'].values[0]

In [12]:
moreThanOne = userLog2VecDf['project_id'].apply(len)>2
projs = userLog2VecDf['project_id']#[moreThanOne]
projs = [[str(p) for p in proj] for proj in projs]
users = userLog2VecDf.index#[moreThanOne]


In [13]:
documents = [TaggedDocument(proj, [user]) for i, (proj,user) in enumerate(zip(projs,users))]

In [14]:
documents[:10]

[TaggedDocument(words=['5342'], tags=['00005aba-5ebc-0821-f5a9-bacca40be125']),
 TaggedDocument(words=['5678'], tags=['0000bae7-6233-d7cc-2a6d-48aa70fe8ad4']),
 TaggedDocument(words=['6461'], tags=['0000c576-e929-19eb-615a-349ec3b4709b']),
 TaggedDocument(words=['9040'], tags=['0000d196-6385-80b8-661d-b7427042daa3']),
 TaggedDocument(words=['6709', '6712'], tags=['0000e1e2-f595-0ae7-860f-fcc07dcb116e']),
 TaggedDocument(words=['8829', '4703', '6577', '4703', '8829', '4184'], tags=['0000fa46-1f0b-9504-b568-43479d17620e']),
 TaggedDocument(words=['6425'], tags=['00011133-6093-88a6-98b8-de44b88170f3']),
 TaggedDocument(words=['4210'], tags=['00011607-0514-1c6c-9e93-30aca362f7e6']),
 TaggedDocument(words=['8159'], tags=['00015aac-868d-57da-d68a-1ff71b598380']),
 TaggedDocument(words=['8587', '5985', '9095'], tags=['00019a11-0aaa-2559-4feb-534724830d3e'])]

### Train

In [15]:
model = Doc2Vec(documents, vector_size=300, window=3, min_count=1, workers=4,epochs=4,sample=1e-4, negative=20)

### Let's see if it works or not

In [ ]:
# def recommendByVector(userId):
#     try:
#     #print(most_similar_user)
#         seen = userLog2VecDf.loc[userId].project_id
#         most_similar_user = model.docvecs.most_similar(userId,topn=100)
#         recommending_projs = []
#         for user, similarity in most_similar_user:
#             friend_projects = [i for i in userLog2VecDf.loc[user].project_id if i not in seen]
#             recommending_projs.extend(friend_projects)
#             recommending_projs = list(set(recommending_projs))
#             if len(recommending_projs)>=7: break

#         #     print(userLog2VecDf.loc[user].project_id, similarity)

#         return " ".join([str(i) for i in recommending_projs])
#     except:
#         return "5858 3738 4201 5452 5678 5764 5067"
    
# recommendByVector("0018fd5f-213f-c9ab-ebf4-0dcfb63be43b")

In [60]:
all_project = model.wv.vocab
def recommendByProjects(userId, userLog=userLog2VecDf,topn = 7):
    seen = userLog.loc[userId].project_id
    
    seen = [str(i) for i in seen if str(i) in all_project]
    if len(seen) ==0: return "5858 3738 4201 5452 5678 5764 5067"
    most_similar_proj = model.wv.most_similar(positive=seen,topn=100)
    recommending_projs = []
    for proj, similarity in most_similar_proj:
        if proj not in seen: recommending_projs.append(proj)
        if len(recommending_projs)>=topn: break

    #     print(userLog.loc[user].project_id, similarity)

    return " ".join([str(i) for i in recommending_projs])

recommendByProjects("0018fd5f-213f-c9ab-ebf4-0dcfb63be43b")

/home/users/u5722792744_1/miniconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'6161 6172 8112 4627 7694 2028 6405'

In [36]:
testUserAnswer = test.groupby("userCode").agg({"project_id":lambda x: list(x)})
testUserAnswer.columns = ["ans_project_list"]
testUserAnswerMerged = userLog2VecDf.merge(testUserAnswer,on='userCode',right_index=True)

In [40]:
testing = testUserAnswerMerged.reset_index()
ans = testing['userCode'].apply(recommendByProjects)
testing['preds'] = ans
testing['ans_project'] = testing['ans_project_list'].apply(lambda x: list(set(x)))
testing['MAP'] = testing.apply(lambda x: score_AP(x['ans_project'],x['preds']),axis=1)
print("MAP@7",testing['MAP'].mean())#0.0296277 by 4 epochs 300dim

/home/users/u5722792744_1/miniconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


MAP@7 0.028017888195096482


In [75]:
scores = []
for i in range(5):
    testing_users = pd.read_csv("./clean_data/test_data/test_before_021718_7000row_fold{}.csv".format(i))
    testing_users.head()

    testing_users['project_id'] = testing_users['userCode'].apply(recommendByProjects)
    testing_users['ans_project_id'] = testing_users['ans_project_id'].apply(eval)
    #testing_users['ans_project'] = testing_users['ans_project_id'].apply(lambda x: list(set(x)))
    testing_users['MAP'] = testing_users.apply(lambda x: score_AP(x['ans_project_id'],x['project_id']),axis=1)
    score = testing_users['MAP'].mean()
    print(score)
    scores.append(score)

/home/users/u5722792744_1/miniconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.04318023080660835
0.042492463556851316
0.04292921282798833
0.04283192743764172
0.04328287495950761


In [51]:
#testing_users = pd.read_csv("./data/testing_users.csv")
#testing_users = pd.read_csv("./data/200_users.csv")
testing_users = pd.read_csv("./clean_data/test_data/test_before_021718_7000row_fold0.csv")
testing_users.head()

testing_users['project_id'] = testing_users['userCode'].apply(recommendByProjects)
testing_users['ans_project_id'] = testing_users['ans_project_id'].apply(eval)
#testing_users['ans_project'] = testing_users['ans_project_id'].apply(lambda x: list(set(x)))
testing_users['MAP'] = testing_users.apply(lambda x: score_AP(x['ans_project_id'],x['project_id']),axis=1)
testing_users['MAP'].mean()

/home/users/u5722792744_1/miniconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.04318023080660835

In [ ]:
print("Accuracy: ",testing['MAP'].apply(lambda x: x>0).sum()/testing.shape[0])

In [39]:
def score_AP(correct, recommended, n=7):
    recommended = recommended.split(" ")
    assert(len(set(recommended))==len(recommended)) # Recommended should be unique    
    precisions = []
    correct_score = 1
    for idx, recommend_item in enumerate(recommended):
        try:
            if int(recommend_item) not in correct: precisions.append(0)
            else:                         
                precisions.append(correct_score/(idx+1))
                correct_score+=1
        except:
            print(recommend_item)
    return sum(precisions)/min(len(correct),n)
#score_AP(["5858", "5678"],"5678 5858 8058 8107 6527 9132 6446",7)

In [ ]:
testing.head().apply(lambda x: score_AP(x['ans_project_list'],x['preds']),axis=1)

In [55]:
submission_user = pd.read_csv("./data/testing_users.csv")
userLogForSubmit = pd.read_csv("./clean_data/userlog_feature.csv")
userLogNewest = userLogForSubmit.groupby("userCode").agg({"project_id":lambda x: list(x)})

In [70]:
submission_user['project_id'] = submission_user['userCode'].apply(lambda x: recommendByProjects(x,userLogNewest))

/home/users/u5722792744_1/miniconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [71]:
submission_user.head()

,userCode,project_id
0,003366c6-0cd9-48e7-d134-2051d7360c2d,6734 6702 6714 6802 6587 8853 6666
1,003b2f18-60b4-37be-c63d-0caead1afa97,8089 6181 7825 4617 4615 7809 6499
2,003d571e-9de7-6de3-2cb6-2ae190cc7beb,8382 6725 6780 8882 6663 6646 6670
3,00426ede-6813-949b-de2e-6d8b228dff1d,8422 5483 4181 6278 3239 8384 2904
4,0043a9c7-cace-1aa1-3188-4b173bf28991,5466 6352 4800 998 8068 5668 4286


In [74]:
submission_user.to_csv("./submission/item2vec_v1.csv",index=False)

In [77]:
model.save("./model/doc2vec_before_train.wv")